<a href="https://colab.research.google.com/github/stinkymatt/pub/blob/master/AstraGeohashSamples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-cloud-secret-manager
!pip install astrapy
!pip install cassandra-driver

In [ ]:
#Credentials for a Read-only user
auth_id = "" #@param {type: "string"}
auth_secret = "" #@param {type:"string"}
auth_token = "" #@param {type: "string"}

ASTRA_DB_ID='c2902cac-68fa-4d5a-91dd-bc8163619c98'
ASTRA_DB_REGION='us-east4'
ASTRA_DB_KEYSPACE='geodemos'

from astrapy.client import create_astra_client
astra_client = create_astra_client(astra_database_id=ASTRA_DB_ID,
                                   astra_database_region=ASTRA_DB_REGION,
                                   astra_application_token=auth_token)
zipUrl = astra_client.ops.get_secure_bundle(database=ASTRA_DB_ID)

import urllib
urllib.request.urlretrieve(zipUrl['downloadURL'], 'scb.zip')

In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

def connect_to_astra():
  cloud_config= {'secure_connect_bundle':'scb.zip'}
  auth_prov = PlainTextAuthProvider(auth_id, auth_secret)
  cluster = Cluster(cloud=cloud_config, auth_provider=auth_prov)
  return cluster.connect()

#Connect to Astra and query from geodemos.my_mapdata table
#There are string matching indexes on geo_hash3, geo_hash4, geo_hash5 and geo_hash10
#The general idea is that you could move easily between the 3/4/5 "zoom levels"
#9xj3t is a crowded 5-character geo_hash to try out. 
#There are 250,000 items in this DB
session = connect_to_astra()

rows = session.execute("select md_pk, md_name, md_lat, md_lng, geo_hash10 from geodemos.my_mapdata where geo_hash5='9xj3t'")
#rows = session.execute("select md_pk, md_name, md_lat, md_lng, geo_hash10 from geodemos.my_mapdata where geo_hash4='9xj3'")
#rows = session.execute("select md_pk, md_name, md_lat, md_lng, geo_hash10 from geodemos.my_mapdata where geo_hash3='9xj'")
rows.all()

#Just for pretty printing below
#import pandas as pd
#from google.colab import data_table
#data_table.enable_dataframe_formatter()
#data = pd.DataFrame(rows) 
#data
